In [13]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import models


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


In [14]:
#directory = "/Users/abhar/Downloads/face_age/face_age_image" 

directory = "/Users/abhar/Downloads/practice_imageset" #practice data set

dataset = tf.keras.utils.image_dataset_from_directory(
        directory,
        labels="inferred",
        label_mode="int",
        class_names=None,
        color_mode="rgb",
        batch_size=16,
        image_size=(200, 200),

)

train_ds = tf.keras.utils.image_dataset_from_directory(directory,validation_split=0.2,subset="training",seed = 42)
validation_ds = tf.keras.utils.image_dataset_from_directory(directory,validation_split=0.2,subset="validation",seed = 42)

#testx_ds = tf.keras.utils.image_dataset_from_directory(directory,validation_split=0.3,subset="validation",seed = 42)
#testy_ds = tf.keras.utils.image_dataset_from_directory(directory,validation_split=0.3,subset="validation",seed = 42)
#print(dataset)

#for i in dataset:
    #print(i)

num_classes = 4

Found 2131 files belonging to 4 classes.
Found 2131 files belonging to 4 classes.
Using 1705 files for training.
Found 2131 files belonging to 4 classes.
Using 426 files for validation.


In [19]:
#Make model
model = Sequential()
model.add(layers.InputLayer(input_shape=(256, 256, 3)))

# 1st conv block
model.add(layers.Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='same'))
model.add(layers.MaxPool2D(pool_size=(2, 2), padding='same'))
# 2nd conv block
model.add(layers.Conv2D(50, (5, 5), activation='relu', strides=(2, 2), padding='same'))
model.add(layers.MaxPool2D(pool_size=(2, 2), padding='same'))
model.add(layers.BatchNormalization())
# 3rd conv block
model.add(layers.Conv2D(70, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model.add(layers.MaxPool2D(pool_size=(2, 2), padding='valid'))
model.add(layers.BatchNormalization())
# ANN block
model.add(layers.Flatten())
model.add(layers.Dense(units=num_classes, activation='relu'))
model.add(layers.Dense(units=num_classes, activation='relu'))
model.add(layers.Dropout(0.25))
# output layer
model.add(layers.Dense(units=num_classes, activation='softmax'))
model.summary()

opt = Adam(learning_rate=1e-3, decay=1e-3 / 200)
model.compile(optimizer='adam',loss="sparse_categorical_crossentropy",metrics=['accuracy'])

# train the model
print("[INFO] training model...")
im_model = model.fit(train_ds,epochs=10,validation_data=validation_ds,batch_size=8)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 256, 256, 25)      1900      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 128, 128, 25)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 64, 50)        31300     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 32, 32, 50)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 50)        200       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 16, 70)        31570     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 8, 8, 70)         

C:\Users\abhar\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


54/54 [==============================] - 220s 4s/step - loss: 1.2795 - accuracy: 0.5097 - val_loss: 2.0805 - val_accuracy: 0.3873
Epoch 2/10
54/54 [==============================] - 148s 3s/step - loss: 1.2096 - accuracy: 0.5337 - val_loss: 1.4726 - val_accuracy: 0.2981
Epoch 3/10
54/54 [==============================] - 146s 3s/step - loss: 1.1637 - accuracy: 0.5343 - val_loss: 1.3065 - val_accuracy: 0.3967
Epoch 4/10
54/54 [==============================] - 179s 3s/step - loss: 1.1583 - accuracy: 0.5343 - val_loss: 2.1654 - val_accuracy: 0.4836
Epoch 5/10
54/54 [==============================] - 189s 3s/step - loss: 1.1247 - accuracy: 0.5337 - val_loss: 1.1849 - val_accuracy: 0.4648
Epoch 6/10
54/54 [==============================] - 176s 3s/step - loss: 1.0930 - accuracy: 0.5343 - val_loss: 1.1253 - val_accuracy: 0.4718
Epoch 7/10
54/54 [==============================] - 157s 3s/step - loss: 1.0843 - accuracy: 0.5343 - val_loss: 1.1338 - val_accuracy: 0.4718
Epoch 8/10
54/54 [======